<a href="https://colab.research.google.com/github/Method-for-Software-System-Development/Cloud_Computing/blob/develop/logic/repair_controller.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Navigate to project folder and ensure we're on the 'develop' branch
import os, subprocess, sys

REPO_DIR = "/content/Cloud_Computing"
if not os.path.isdir(REPO_DIR):
    subprocess.run([
        "git", "clone", "-b", "develop",
        "https://github.com/Method-for-Software-System-Development/Cloud_Computing.git",
        REPO_DIR
    ], check=True)

# Update & checkout develop if repo already exists
subprocess.run(["git", "-C", REPO_DIR, "fetch", "origin"], check=True)
subprocess.run(["git", "-C", REPO_DIR, "checkout", "develop"], check=True)
subprocess.run(["git", "-C", REPO_DIR, "pull"], check=True)

# Install importnb (only needed once per session)
%pip install importnb --quiet

# Add firebase folder to Python path and import its notebook as module
sys.path.append(os.path.join(REPO_DIR, "firebase"))
sys.path.append(os.path.join(REPO_DIR, "logic"))
from importnb import Notebook

with Notebook():
    import FireBase as fb
    import user_controller as uc

from datetime import datetime



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 1.9 MB/s eta 0:00:00
omer_cohen 050-111-2222 200
shira_gold 050-222-3333 200
daniel_bar 050-333-4444 200
roni_benami 050-444-5555 200
lior_levi 050-555-6666 200
yael_elyashiv 050-666-7777 200
noa_katz 050-777-8888 200
matan_tal 050-888-9999 200
or_peled 050-999-0000 200
tom_segal 050-000-1111 200
Daniel Bar | 050-333-4444 | Robotics Engineer
Lior Levi | 050-555-6666 | Backend Developer
Matan Tal | 050-888-9999 | Mechanical Engineer
Noa Katz | 050-777-8888 | UX Designer
Omer Cohen | 050-111-2222 | Automation Engineer
Or Peled | 050-999-0000 | System Engineer
Roni Ben Ami | 050-444-5555 | QA Engineer
Shira Gold | 050-222-3333 | Electrical Engineer
Tom Segal | 050-000-1111 | Production Engineer
Yael Elyashiv | 050-666-7777 | Hardware Engineer
Indoor Sensor Data:
Status: 200 | Uploaded: {'Temperature': 26.4, 'Humidity': 41.0, 'Pressure': 974.1, 'Distance': 210.1}
Status: 200 | Uploaded: {'Temperature': 30.2, 'Humidity': 28.7, 'Pressu

In [3]:
def start_repair(sensor: str):
    """
    Start the repair process for a given sensor.
    Logs the start time and updates the fault status in Firebase.
    """
    code, fault = fb.get_fault(sensor)
    if code != 200 or fault is None:
        return f"Fault '{sensor}' not found."

    now_iso = datetime.now().isoformat()
    code, res = fb.update_fault(sensor, {
        "repair_start_time": now_iso,
        "status": "Repair Started"
    })
    if code != 200:
        return f"Failed to start repair for '{sensor}'."

    return f"Repair for '{sensor}' started at {now_iso}."


def calculate_fault_score(fault):
    """
    Calculate score based on severity and repair speed.
    Base scores:
        Low = 50
        Medium = 100
        High = 200
    Additional score = higher if repaired faster.
    """

    severity_multipliers = {
        "Low": 50,
        "Medium": 100,
        "High": 200
    }

    severity = fault.get("severity", "Low")
    multiplier = severity_multipliers.get(severity, 50)

    try:
        start_time = datetime.fromisoformat(fault["repair_start_time"])
        end_time = datetime.now()
        minutes_passed = max(1, (end_time - start_time).total_seconds() / 60)
    except Exception:
        minutes_passed = 1
    additional_score = min(60, 60 * (60 / minutes_passed))

    total_score = multiplier + round(additional_score)
    return total_score


def complete_repair(sensor: str):
    """
    Completes the repair process for a fault:
    - Retrieves the currently logged-in user.
    - Calculates XP.
    - Updates fault status in Firebase.
    - Updates the user's XP.

    Args:
        sensor (str): The ID of the sensor to complete repair for.

    Returns:
        str: Summary message about the repair and score.
    """
    code, user = uc.get_current_user()
    if code != 200:
        return "❌ No user is logged in."

    username = user["username"]

    code, fault = fb.get_fault(sensor)
    if code != 200 or fault is None:
        return f"❌ Fault '{sensor}' not found."

    score = calculate_fault_score(fault)

    # Mark fault as repaired and assign it to the current user
    fb.mark_fault_as_repaired(sensor, username)

    # Update user's XP in Firebase
    code, response = fb.update_user_score(username, score)
    if code != 200:
        return f"❌ Failed to update user score for '{username}'."

    # Retrieve updated user data to show new score
    code, user_data = fb.get_user(username)
    if code != 200 or user_data is None:
        return f"✅ Fault repaired, but failed to retrieve updated user score."

    new_score = user_data.get("score", "?")
    return f"✅ Fault '{sensor}' repaired by {username}. +{score} XP (Total: {new_score})."
